In [280]:
###Dependecies
import sqlite3
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np

In [4]:
steam_games_df = pd.read_csv("db_files/steam_games.csv", encoding = 'utf-8', dtype={'name':str})
steam_games_df.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


In [260]:
#remove uneeded rows
#delete_labels = ['url', 'types', 'desc_snippet', 'popular_tags', 'game_details', 'languages', 'achievements',
#                 'game_description', 'mature_content', 'minimum_requirements', 'recommended_requirements', 'discount_price']
#steam_games_df = steam_games_df.drop(columns=delete_labels)
steam_games_df.rename(columns = {'name': 'Game Title',
                                 'recent_reviews': 'Recent Reviews',
                                 'all_reviews': 'All Reviews',
                                 'release_date': 'Release Date',
                                 'developer': 'Developer',
                                 'publisher': 'Publisher',
                                 'genre': 'Genre',
                                 'original_price': 'Original Price'
                                }, inplace = True)
steam_games_df.head()

,Game Title,Recent Reviews,All Reviews,Release Date,Developer,Publisher,Genre,Original Price
0,DOOM,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks",Action,$19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Action,Adventure,Massively Multiplayer",$29.99
2,BATTLETECH,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Action,Adventure,Strategy",$39.99
3,DayZ,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Action,Adventure,Massively Multiplayer",$44.99
4,EVE Online,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Action,Free to Play,Massively Multiplayer,RPG,...",Free


In [284]:
# sort the Game Title column into boolean so I can use iloc to shrink the dataset to match
# the game titles I pulled from kicksarter
temp_list = steam_games_df['Game Title'].map(lambda x: True if x in game_names else False)
indicies = np.where(temp_list)[0]

236
[   14    60   157   194   252   274   288   290   316   354   371   428
   483   524   571   597   618   650   737   743   746   808   830   888
   941   976  1014  1056  1150  1222  1230  1239  1250  1489  1499  1597
  1598  1604  1691  1707  1748  1758  1795  1829  1953  1975  1998  2039
  2075  2172  2192  2261  2262  2378  2394  2434  2437  2489  2588  2626
  2642  2705  2745  2785  2867  3003  3017  3018  3110  3141  3186  3194
  3340  3411  3482  3507  3629  3737  3782  3814  3849  4010  4060  4076
  4090  4155  4179  4187  4331  4378  4571  4723  4736  4824  4826  4902
  5433  5586  5742  5793  6090  6577  7083  7451  7508  7565  8228  8392
  8663  8754  9045  9184  9421 13068 15529 17695 17810 18277 18321 18976
 19011 19484 19564 20185 20212 20641 20715 21031 21044 21110 21141 21151
 21443 21490 21496 21512 22295 22368 22578 22853 23554 23670 24246 24278
 25063 25542 26319 26491 26624 26828 27049 27077 27087 27134 27183 27324
 27364 27637 28081 28253 28263 28264 28505 2899

In [286]:
steam_games_sorted_df = pd.DataFrame()
steam_games_sorted_df = steam_games_df.iloc[indicies]
steam_games_sorted_df = steam_games_sorted_df.set_index('Game Title')
steam_games_sorted_df

,Recent Reviews,All Reviews,Release Date,Developer,Publisher,Genre,Original Price
Game Title,,,,,,,
Stonehearth,"Mixed,(66),- 40% of the 66 user reviews in the...","Mostly Positive,(5,484),- 75% of the 5,484 use...","Jul 25, 2018",Radiant Entertainment,"(none),(none)","Indie,Simulation,Strategy",$19.99
Pillars of Eternity,"Very Positive,(60),- 85% of the 60 user review...","Very Positive,(8,015),- 84% of the 8,015 user ...","Mar 26, 2015",Obsidian Entertainment,"Paradox Interactive,Paradox Interactive",RPG,$59.98
Town of Salem,"Mostly Positive,(208),- 71% of the 208 user re...","Very Positive,(13,158),- 86% of the 13,158 use...","Dec 15, 2014",BlankMediaGames,"BlankMediaGames,BlankMediaGames","Indie,RPG,Strategy",$4.99
Fell Seal: Arbiter's Mark,"Very Positive,(122),- 94% of the 122 user revi...","Very Positive,(725),- 92% of the 725 user revi...","Apr 30, 2019",6 Eyes Studio,"1C Entertainment,1C Entertainment","Indie,RPG,Strategy",$29.99
Elite Dangerous,"Mixed,(198),- 66% of the 198 user reviews in t...","Mixed,(28,383),- 68% of the 28,383 user review...","Apr 2, 2015",Frontier Developments,"Frontier Developments,Frontier Developments","Action,Adventure,Massively Multiplayer,RPG,Sim...",$29.99
...,...,...,...,...,...,...,...
StarCrossed,NaN,NaN,2019,Contigo Games,"Whitethorn Digital,Whitethorn Digital","Action,Indie",NaN
Gibbous - A Cthulhu Adventure,NaN,NaN,Soon!,Stuck In Attic,"Stuck In Attic,Stuck In Attic","Adventure,Indie",NaN
Children of Morta,NaN,NaN,Summer 2019 - wishlist now!,Dead Mage,"11 bit studios,11 bit studios","Action,Adventure,Indie,RPG",NaN


In [287]:
final_steam_df = steam_games_sorted_df
final_steam_df

,Recent Reviews,All Reviews,Release Date,Developer,Publisher,Genre,Original Price
Game Title,,,,,,,
Stonehearth,"Mixed,(66),- 40% of the 66 user reviews in the...","Mostly Positive,(5,484),- 75% of the 5,484 use...","Jul 25, 2018",Radiant Entertainment,"(none),(none)","Indie,Simulation,Strategy",$19.99
Pillars of Eternity,"Very Positive,(60),- 85% of the 60 user review...","Very Positive,(8,015),- 84% of the 8,015 user ...","Mar 26, 2015",Obsidian Entertainment,"Paradox Interactive,Paradox Interactive",RPG,$59.98
Town of Salem,"Mostly Positive,(208),- 71% of the 208 user re...","Very Positive,(13,158),- 86% of the 13,158 use...","Dec 15, 2014",BlankMediaGames,"BlankMediaGames,BlankMediaGames","Indie,RPG,Strategy",$4.99
Fell Seal: Arbiter's Mark,"Very Positive,(122),- 94% of the 122 user revi...","Very Positive,(725),- 92% of the 725 user revi...","Apr 30, 2019",6 Eyes Studio,"1C Entertainment,1C Entertainment","Indie,RPG,Strategy",$29.99
Elite Dangerous,"Mixed,(198),- 66% of the 198 user reviews in t...","Mixed,(28,383),- 68% of the 28,383 user review...","Apr 2, 2015",Frontier Developments,"Frontier Developments,Frontier Developments","Action,Adventure,Massively Multiplayer,RPG,Sim...",$29.99
...,...,...,...,...,...,...,...
StarCrossed,NaN,NaN,2019,Contigo Games,"Whitethorn Digital,Whitethorn Digital","Action,Indie",NaN
Gibbous - A Cthulhu Adventure,NaN,NaN,Soon!,Stuck In Attic,"Stuck In Attic,Stuck In Attic","Adventure,Indie",NaN
Children of Morta,NaN,NaN,Summer 2019 - wishlist now!,Dead Mage,"11 bit studios,11 bit studios","Action,Adventure,Indie,RPG",NaN


In [292]:
#complete same process with the metacritic csv from kaggle
metacritic_df = pd.read_csv("db_files/metacritic_games.csv", encoding = 'utf-8')
delete_labels = ['platform', 'number_players', 'rating']
clean_metacritic_df = metacritic_df.drop(columns=delete_labels)
clean_metacritic_df = clean_metacritic_df.rename(columns={'game': 'Game Title',
                              'developer': 'Developer',
                              'genre': 'Genre',
                              'release_date': 'Release Date',
                              'positive_critics': 'Positive Critics',
                              'neutral_critics': 'Neutral Critics',
                              'negative_critics': 'Negative Critcs',
                              'positive_users': 'Positive Users',
                              'neutral_users': 'Neutral Users',
                              'negative_users': 'Negative Users',
                              'metascore': 'Metascore',
                              'user_score': 'User Score'                             
                             })
clean_metacritic_df

,Game Title,Developer,Genre,Release Date,Positive Critics,Neutral Critics,Negative Critcs,Positive Users,Neutral Users,Negative Users,Metascore,User Score
0,Portal 2,Valve Software,Action,"Apr 18, 2011",51,1,0,1700,107,19,95,90
1,The Elder Scrolls V: Skyrim,Bethesda Game Studios,Role-Playing,"Nov 10, 2011",32,0,0,1616,322,451,94,82
2,The Legend of Zelda: Ocarina of Time 3D,GREZZO,Miscellaneous,"Jun 19, 2011",84,1,0,283,20,5,94,90
3,Batman: Arkham City,Rocksteady Studios,Action Adventure,"Nov 21, 2011",27,0,0,240,34,27,91,87
4,Super Mario 3D Land,Nintendo,Action,"Nov 13, 2011",81,1,0,251,39,11,90,84
...,...,...,...,...,...,...,...,...,...,...,...,...
5694,Modern Combat: Blackout,Gameloft,Action,"Feb 14, 2019",0,5,2,1,0,1,53,57
5695,Fimbul,Zaxis,Action Adventure,"Feb 28, 2019",0,7,2,1,0,0,52,18
5696,Generation Zero,Avalanche Studios,Action Adventure,"Mar 26, 2019",1,11,6,5,2,6,50,57
5697,Vane,Friend & Foe,Adventure,"Jan 15, 2019",3,26,22,2,2,2,49,55


In [295]:
#metacritic_df.loc[metacritic_df['platform']=='PC']
temp_list = clean_metacritic_df['Game Title'].map(lambda x: True if x in game_names else False)
indicies = np.where(temp_list)[0]
print(indicies)
print(len(indicies))

[  23   77  143  313  374  406  544  716  801  818  830  834  871  883
  909  915  978 1015 1031 1062 1067 1076 1110 1209 1229 1284 1375 1381
 1463 1469 1476 1489 1533 1538 1555 1560 1597 1613 1628 1641 1643 1649
 1708 1710 1717 1777 1834 1842 1908 2084 2091 2114 2136 2161 2162 2195
 2255 2259 2263 2269 2376 2407 2434 2509 2520 2521 2540 2551 2695 2710
 2723 2734 2747 2763 2786 2804 2825 2826 2871 2933 2942 2950 3038 3041
 3053 3081 3102 3117 3236 3268 3282 3377 3384 3387 3388 3394 3424 3436
 3454 3481 3482 3491 3496 3500 3504 3522 3539 3556 3562 3672 3704 3729
 3758 3872 3893 3906 3924 3986 3987 4011 4036 4058 4161 4193 4239 4301
 4312 4330 4419 4474 4541 4568 4588 4694 4717 4771 4845 4874 5015 5054
 5067 5092 5132 5180 5259 5270 5274 5302 5345 5359 5426 5489 5565 5664]
154


In [301]:
final_metacritic_df = pd.DataFrame()
final_metacritic_df = clean_metacritic_df.iloc[indicies]
final_metacritic_df = final_metacritic_df.set_index('Game Title')
final_metacritic_df

,Developer,Genre,Release Date,Positive Critics,Neutral Critics,Negative Critcs,Positive Users,Neutral Users,Negative Users,Metascore,User Score
Game Title,,,,,,,,,,,
SpaceChem,Zachtronics Industries,Miscellaneous,"Mar 2, 2011",11,2,0,45,3,2,84,85
Xenonauts,"Gamethunks, Goldhawk Interactive",General,"Nov 4, 2011",15,5,1,30,9,8,77,79
Before the Echo,Iridium Studios,Action,"Oct 20, 2011",3,4,0,17,8,3,70,74
FTL: Faster Than Light,Subset Games,Simulation,"Sep 14, 2012",41,1,0,273,35,55,84,84
Chivalry: Medieval Warfare,Torn Banner Studios,Action,"Oct 16, 2012",19,5,0,189,64,56,79,78
...,...,...,...,...,...,...,...,...,...,...,...
InnerSpace,PolyKnight Games,Simulation,"Jan 16, 2018",2,15,0,1,0,0,63,68
Road Redemption,Dark Seas Games,Simulation,"Nov 6, 2018",1,12,0,2,0,0,59,44
Desert Child,Oscar Brittain,Racing,"Dec 11, 2018",0,5,2,0,2,0,47,50


In [8]:
kickstarter_df = pd.read_csv("db_files/ks-projects-201801.csv", encoding = 'utf-8')
kickstarter_df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [9]:
#considering taking this out and doing more web scraping to get relevant values

#sort kickstarter df for just video games
ks_games_df = kickstarter_df.loc[kickstarter_df['category']=='Video Games']
#sort for successful ks campaigns
ks_games_df = ks_games_df.loc[ks_games_df['state'] == 'successful']
ks_games_df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
126,1000524949,Operation: Make Stuff,Video Games,Games,USD,2012-10-18,200.0,2012-09-18 04:46:06,306.72,successful,36,US,306.72,306.72,200.00
199,1000811882,The Sword of Asumi Visual Novel,Video Games,Games,GBP,2014-11-10,2500.0,2014-10-19 20:35:23,3097.00,successful,107,GB,4983.78,4848.23,3913.65
219,1000940032,Space Trucker,Video Games,Games,USD,2016-12-08,150.0,2016-11-08 08:20:27,165.00,successful,7,US,25.00,165.00,150.00
319,1001442379,The Contractors - Create 3D Model: Prop Asset ...,Video Games,Games,USD,2017-05-26,2000.0,2017-05-05 12:47:05,2348.00,successful,17,US,340.00,2348.00,2000.00
475,1002272545,Course: Make a Stunning Fantasy RPG with GameM...,Video Games,Games,USD,2016-08-13,10200.0,2016-07-14 00:01:11,20709.50,successful,1258,US,11973.50,20709.50,10200.00


In [322]:
#complete same process for ign csv from kaggle
ign_df = pd.read_csv("db_files/IGN_reviews.csv", encoding = 'utf-8')
clean_ign_df = ign_df.rename(columns={'Game': 'Game Title',
                                      'Score': 'IGN Game Score'})


In [325]:
temp_list = clean_ign_df['Game Title'].map(lambda x: True if x in game_names else False)
indicies = np.where(temp_list)[0]
print(indicies)

#Unfortunate that there's only 14 values from IGN, I suspect there are more but this particular dataset may be out of date
clean_ign_df = clean_ign_df.iloc[indicies]
clean_ign_df = clean_ign_df.loc[clean_ign_df['Platform']=='PC']
clean_ign_df

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


,Game Title,Platform,IGN Game Score,Genre
4,Sportsfriends,PC,8.7,"Action, Compilation"
14,Tesla Effect: A Tex Murphy Adventure,PC,6.8,Adventure
39,FTL: Faster Than Light,PC,9.6,Simulation
125,Octodad: Dadliest Catch,PC,7.8,Adventure
143,The Banner Saga,PC,8.6,"Strategy, RPG"
342,Legends of Aethereus,PC,4.7,"Action, RPG"
371,Takedown: Red Sabre,PC,4.0,Shooter
423,Guncraft,PC,6.4,Shooter
501,Shadowrun Returns,PC,7.3,RPG
746,Akaneiro: Demon Hunters,PC,4.5,RPG


In [326]:
final_ign_df = clean_ign_df.set_index('Game Title')
final_ign_df

,Platform,IGN Game Score,Genre
Game Title,,,
Sportsfriends,PC,8.7,"Action, Compilation"
Tesla Effect: A Tex Murphy Adventure,PC,6.8,Adventure
FTL: Faster Than Light,PC,9.6,Simulation
Octodad: Dadliest Catch,PC,7.8,Adventure
The Banner Saga,PC,8.6,"Strategy, RPG"
Legends of Aethereus,PC,4.7,"Action, RPG"
Takedown: Red Sabre,PC,4.0,Shooter
Guncraft,PC,6.4,Shooter
Shadowrun Returns,PC,7.3,RPG


In [128]:
#Now to scrape the kickstarter page for successful games that launched on steam
#looking for the names of each game and potentially other info that might be interesting to add to the db
url='https://www.kickstarter.com/play?sort=alpha#all-projects'
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\87.0.4280.88]


In [153]:
#set initial url and retrieve html for the first page
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#list for storing the links to the games' corresponding steam pages
steam_links=[]
counter = 0

#while loop executes as long as the new page contains a "view more" button
#This means it will not collect links from the final page, which doesn't have the button
while soup.find('a', class_='more btn btn--border-gray', href=True) is not None:
    
    #inital .find for the html elements
    ks_pages = soup.find('section', {'id' : 'all-projects'})
    
    #find the class with the steam link and save it
    for a in ks_pages.find_all('a', class_= 'hover-zoomout f3 bg-green-90 track-service', href=True):
        try:
            steam_links.append(a['href'])
        except KeyError:
            pass
    #check for more pages, browser.visit takes us to the next page if it's there
    try:
        button = soup.find('a', class_='more btn btn--border-gray', href=True)['href']
        browser.visit(button)
        counter += 1
        print(f"clicks: {counter}")
        time.sleep(1)
    except TypeError:
        pass
    
    #reset html and soup for the new page
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
#for the final page that does not have the view more button
ks_pages = soup.find('section', {'id' : 'all-projects'})
    
#find the class with the steam link and save it
for a in ks_pages.find_all('a', class_= 'hover-zoomout f3 bg-green-90 track-service', href=True):
    try:
        steam_links.append(a['href'])
    except KeyError:
        pass
    
#check that we retrieved all the links
len(steam_links)

clicks: 1
clicks: 2
clicks: 3
clicks: 4
clicks: 5
clicks: 6
clicks: 7
clicks: 8
clicks: 9
clicks: 10
clicks: 11
clicks: 12
clicks: 13
clicks: 14
clicks: 15
clicks: 16


255

In [164]:
#now to scrape steam for the name of the game
length = len(steam_links)
game_names = []
for i in range(length):
    browser.visit(steam_links[i])
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        game_names.append(soup.find(class_='apphub_AppName').text)
    except:
        print(f"unexpected error: {steam_links[i]} index: {i}")
    time.sleep(1)

unexpected error: https://store.steampowered.com/app/249380 index: 22
unexpected error: https://store.steampowered.com/app/210870 index: 35
unexpected error: https://store.steampowered.com/app/258030 index: 58
unexpected error: https://store.steampowered.com/app/299700 index: 64
unexpected error: https://store.steampowered.com/app/201040 index: 74
unexpected error: https://store.steampowered.com/app/379430 index: 104
unexpected error: https://store.steampowered.com/app/234980 index: 110
unexpected error: https://store.steampowered.com/app/280440 index: 121
unexpected error: https://store.steampowered.com/app/218820 index: 127
unexpected error: https://store.steampowered.com/app/449830 index: 133
unexpected error: https://store.steampowered.com/app/246400 index: 134
unexpected error: https://store.steampowered.com/app/233250 index: 160
unexpected error: https://store.steampowered.com/app/96300 index: 165
unexpected error: https://store.steampowered.com/app/266550 index: 194
unexpected e

In [ ]:
# The errors I found were links to the steam homepage rather than to a specific game.
# There could be a number of reasons why these games are no longer available on steam.
# In the first case with the game 'Carmageddon: Reincarnation' (index[22]) steam still offered other titles 
# from the carmageddon series and it's possible the game is now bundled with other games or is no longer available under the 
# same title listed on kickstarter.

# In another instance, I noticed the correct page was not being pulled up because it required the user to verify their
# age due to graphic content.

In [165]:
browser.visit(steam_links[75])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

soup.find(class_='apphub_AppName').text

'Garden Paws'

In [175]:
print(game_names[22])
#len(game_names)
# the proper steam link for "Castle Story" is at steam_links[23] because steam_link[22] didn't return a game name
print(steam_links[23])

copy_steam_links = steam_links
print(len(steam_links))
print(len(copy_steam_links))
print(copy_steam_links[23])

Castle Story
https://store.steampowered.com/app/227860
255
255
https://store.steampowered.com/app/227860


In [172]:
lost_link_indexes = [22, 35, 58, 64, 74, 104, 110, 121, 127, 133, 134, 160, 165, 194, 196, 202, 221, 237, 250]
#adjust for changing the length of the steam_links list

index_adjust = 0
for i in range(len(lost_link_indexes)):
    lost_link_indexes[i] = lost_link_indexes[i]-index_adjust
    index_adjust +=1
print(lost_link_indexes)



[22, 34, 56, 61, 70, 99, 104, 114, 119, 124, 124, 149, 153, 181, 182, 187, 205, 220, 232]


In [176]:
# careful, running this code multiple times will cause problems
list_to_verify=[]
length = len(lost_link_indexes)
#for i in range(length):
#    list_to_verify.append(copy_steam_links[lost_link_indexes[i]])
#    copy_steam_links.pop(lost_link_indexes[i])

In [263]:
# Finally create dataframe containing the names of games scraped from kickstarter and their steam store urls
webscraped_df = pd.DataFrame(columns=['Game Title', 'Steam Store Url'])
webscraped_df['Game Title'] = game_names
webscraped_df['Steam Store Url'] = copy_steam_links
webscraped_df

,Game Title,Steam Store Url
0,7 Days to Die,https://store.steampowered.com/app/251570
1,A Wizard's Lizard,https://store.steampowered.com/app/280040
2,Adventurezator: When Pigs Fly,https://store.steampowered.com/app/300280
3,Akaneiro: Demon Hunters,https://store.steampowered.com/app/230310
4,All Walls Must Fall - A Tech-Noir Tactics Game,https://store.steampowered.com/app/628290
...,...,...
231,WARMACHINE: Tactics,https://store.steampowered.com/app/253510
232,Whispering Willows,https://store.steampowered.com/app/288060
233,Windforge,https://store.steampowered.com/app/266170
234,Wings! Remastered Edition,https://store.steampowered.com/app/320840


In [274]:
webscraped_df.dtypes

Game Title         object
Steam Store Url    object
dtype: object